# Building RAG applications using Open Source models

## 1. Preparing the local model

### 1-1. Install `Ollama` 

To install `Ollama` on your laptop, use the instruction for [installation](https://ollama.com/download).

After installing it, you can search on its [website](https://ollama.com/) to learn which LLMs are available. Knowing the model, you can then download it through the following command in the terminal window:

```
ollama pull <model_name>
```
Note that here we use `llama2` model. 

You can then check what models you have on your laptop through:

```
ollama list
```

### 1-2. Install `langchain_ollama`

As we want to use LangChain to construct the RAG application, we should install the related package (`langchain_ollama`) using `pip`. 

In [8]:
# !pip install -qU langchain_ollama
from langchain_ollama import OllamaLLM

In [9]:
llm = OllamaLLM(model="llama2")

# to test the model
llm.invoke("The first man on the moon was ...")

'\nThe first man on the moon was Neil Armstrong. He stepped foot on the lunar surface on July 20, 1969, as part of the Apollo 11 mission. Armstrong famously declared "That\'s one small step for man, one giant leap for mankind" as he became the first person to walk on the moon.'

## 2. Create a database

Now, we need to create a database containing the vector embeddings of different part of the document. In order to do that, we need to do the following steps:
* Load the document
* Split it into smaller chunk
* Create the vector embeddings of different chunk and save it in a database

### 2-1. Load the document

In [13]:
# !pip install pypdf
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader #WebBaseLoader

In [29]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=50,
)
# loader = WebBaseLoader('https://en.wikipedia.org/wiki/Albert_Einstein')
loader = PyPDFLoader('data/caselaw_1.pdf') # if you have a pdf document
documents = loader.load_and_split(text_splitter)
print(f"There are {len(documents)} chunks.")
# documents[0]

There are 15 chunks.


### 2-2. Save documents in a Vector Store

In order to extract relevant information (from the documents) to a question, we need to 
* save vector embeddings of documents in a vector store,
* create a retriever which can extract the most relevant chunks to a 

In [18]:
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

In [30]:
vector_store = DocArrayInMemorySearch.from_documents(
    documents,
    embedding=OllamaEmbeddings(model="llama2"),
)

In [49]:
retriever = vector_store.as_retriever()
res = retriever.invoke("which articles have been addressed?")
res[0].page_content[100:]

'4.  On 11 November 2011 the police opened a criminal case under \nArticle 122 (infliction of bodily injuries of medium severity) of the Criminal \nCode. Following the investigations conducted into the case, the case file was \nsent to a court, P. being charged under Article 121 (infliction of grave \ninjuries) of the Criminal Code.'

## 3. Create the RAG application

In [50]:
from langchain.prompts import PromptTemplate

In [51]:
# defining the prompt template
prompt_template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(prompt_template)
print(prompt.format(context='Some context', question='here is the question'))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Some context

Question: here is the question



In [52]:
chain = prompt | llm
chain.input_schema.schema()

/tmp/ipykernel_21072/3173368601.py:2: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  chain.input_schema.schema()


{'properties': {'context': {'title': 'Context', 'type': 'string'},
  'question': {'title': 'Question', 'type': 'string'}},
 'required': ['context', 'question'],
 'title': 'PromptInput',
 'type': 'object'}

In [53]:
# As it can be seen in 'required', the input should contain 'context' and 'question'
chain.invoke({
    'context': 'We are talking about Albert Einstein.',
    'question': 'What subjects was he good at? Give me at least three subjects.'
})

' Of course! Albert Einstein was a brilliant physicist and mathematician, but he was also skilled in other areas. Here are three subjects that he was particularly talented in:\n\n1. Mathematics: Einstein was known for his exceptional mathematical abilities, particularly in the area of differential equations and relativity. He often used mathematical models to describe his theories of physics, and his work in this area laid the foundation for modern mathematics.\n2. Physics: As mentioned earlier, Einstein is best known for his work in physics, specifically his theory of relativity. However, he also made significant contributions to other areas of physics, including thermodynamics, electromagnetism, and quantum mechanics.\n3. Language: Einstein was fluent in several languages, including German, English, French, and Italian. He was known to be particularly skilled in his native language, German, and he often used his knowledge of language to help him communicate complex scientific ideas m

In [54]:
from operator import itemgetter

In [55]:
# Now we create a chain which can reply our questions based on the context
chain = (
    {
        'context': itemgetter('question') | retriever, # here we send the query to the retriever to extract the most relevant chunks (or documents)
        'question': itemgetter('question')
    }
    | prompt
    | llm
)

In [58]:
chain.invoke({
    'question':"Which country was the respondent?"
})

'Based on the context provided, the country that was respondent in the case is Ukraine.'

In [59]:
chain.invoke({
    'question': 'when was the judgment date for this case law?'
})

'Based on the context provided, the judgment date for this case law is:\n\n11 November 2011\n\nThis is indicated in the passage as "On 11 November 2011 the police opened a criminal case under Article 122 (infliction of bodily injuries of medium severity) of the Criminal Code."'

In [60]:
chain.invoke({
    'question': 'which articles from ECHR have been addressed in this case law?'
})

'Based on the provided case law, the following ECHR articles have been addressed:\n\n* Article 3 of the Convention - procedural limb (para. 21)\n* Article 41 of the Convention (para. 5)'

In [62]:
chain.invoke({
    'question': 'Give me the list of all previous cases cited by this case law in the json format?'
})

"I'm happy to help! However, I don't have access to external documents or websites, so I can't answer your question based on the provided context. The case law appears to be referring to specific previous cases, but I don't have the information to generate a list of those cases in JSON format.\n\nIf you provide me with more context or information about the case law, such as the names of the cases or the relevant pages, I may be able to help you better."